In [3]:
import pandas as pd
import json as json

In [2]:
df = pd.read_csv(r"C:\Users\jc4673\Documents\Data\NETS2014_Categories_food.csv")

In [6]:
df[(df['BDS']==1) & (df['GRY']==1)]

,BEH_ID,BDS,GRY,SMK


In [7]:
df[(df['BDS']==1) & (df['SMK']==1)]

,BEH_ID,BDS,GRY,SMK


In [8]:
df[(df['SMK']==1) & (df['GRY']==1)]

,BEH_ID,BDS,GRY,SMK


## Compile List of Non-Overlapping Categories

In [4]:
json_file = r"C:\Users\jc4673\Documents\NETS\config\json_config_2018_07_18.json"
with open(json_file, "r") as f:
    cats = json.load(f)

sic_cats = []
for key in cats:
    if cats[key]["conditional"] in [3, 4, 8]:
        sic_cats.append(key)

In [5]:
df = pd.read_csv(r"C:\Users\jc4673\Documents\NETS\data\NETS2014_WRANGLED\NETS2014_ALL_CATEGORIES_FINAL.csv",
                usecols=['BEH_ID']+sic_cats, index_col=['BEH_ID'], chunksize=5*10**6)

In [6]:
chunk = df.get_chunk()

C:\Users\jc4673\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\numpy\lib\arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
sum_ = chunk.apply(sum, axis=1)

In [8]:
chunk['sum'] = sum_

In [9]:
bad = chunk[chunk['sum'] == 2]
col_sums = bad.apply(sum, axis=0)

In [39]:
bad.loc[bad['OWR'] == 1, ['OWR', 'LAU']]

,OWR,LAU
BEH_ID,,
13001869882,1,1
12001869882,1,1
11001869882,1,1
10001869882,1,1
10001993005,1,1
14002023104,1,1
13002023104,1,1
12002023104,1,1
11002023104,1,1


In [10]:
col_sums[col_sums > 0]

VPA     7032
MPA     7356
CMP     7356
CVP     7032
LAU       65
OWR       65
sum    28906
dtype: int64

In [11]:
chunk['VPA'].sum()

7937

In [12]:
chunk['MPA'].sum()

12351

In [13]:
chunk['LAU'].sum()

11981

In [14]:
chunk['OWR'].sum()

507493

# Track Down the Problematic Codes

In [17]:
def to_zip(iterable):
    # iterable = [int(i) for i in iterable if i]  #convert the iterable to int if it isn't empty str
    return tuple(zip(iterable[0::2], iterable[1::2]))

In [31]:
LAU = cats['LAU']['sic_exclusive'] + \
[item for sublist in [list(range(*x)) for x in to_zip(cats['LAU']['sic_range'])] for item in sublist]

OWR = cats['OWR']['sic_exclusive'] + \
[item for sublist in [list(range(*x)) for x in to_zip(cats['OWR']['sic_range'])] for item in sublist]

In [32]:
len(LAU)

20306

In [33]:
len(OWR)

521249

Overlap looks to be the range: (72199900, 72199999)

In [37]:
list(set(LAU) & set(OWR))

[72199901,
 72199902,
 72199903,
 72199904,
 72199905,
 72199906,
 72199907,
 72199908,
 72199909,
 72199910,
 72199911,
 72199912,
 72199913,
 72199914,
 72199915,
 72199916,
 72199917,
 72199918,
 72199919,
 72199920,
 72199921,
 72199922,
 72199923,
 72199924,
 72199925,
 72199926,
 72199927,
 72199928,
 72199929,
 72199930,
 72199931,
 72199932,
 72199933,
 72199934,
 72199935,
 72199936,
 72199937,
 72199938,
 72199939,
 72199940,
 72199941,
 72199942,
 72199943,
 72199944,
 72199945,
 72199946,
 72199947,
 72199948,
 72199949,
 72199950,
 72199951,
 72199952,
 72199953,
 72199954,
 72199955,
 72199956,
 72199957,
 72199958,
 72199959,
 72199960,
 72199961,
 72199962,
 72199963,
 72199964,
 72199965,
 72199966,
 72199967,
 72199968,
 72199969,
 72199970,
 72199971,
 72199972,
 72199973,
 72199974,
 72199975,
 72199976,
 72199977,
 72199978,
 72199979,
 72199980,
 72199981,
 72199982,
 72199983,
 72199984,
 72199985,
 72199986,
 72199987,
 72199988,
 72199989,
 72199990,
 72199991,